# Audio Transcription Pipeline (AssemblyAI Integration)

Key Tasks:
1. **Transcribe Santa's Story**: Convert an audio file to text using AssemblyAI's transcription service
2. **Create a Summary**: Generate a concise version of the story for quick reading
3. **Adapt for Gen Z**: Transform the story into modern, youth-friendly language using an LLM

The goal is to make Santa's story accessible to different audiences by leveraging AI technologies and the Haystack framework.

## Prerequisites
- AssemblyAI API key
- OpenAI API key (or alternative LLM)
- Basic understanding of Python and Haystack pipelines

## Resources
- [Haystack docs - AssemblyAI integration](https://haystack.deepset.ai/integrations/assemblyai)
- [Haystack docs - Creating pipelines](https://docs.haystack.deepset.ai/docs/creating-pipelines)

## Task 1: Transcribing Santa's Story


### Setup

In [2]:
!uv pip install haystack-ai
!uv pip install assemblyai-haystack
!uv pip install -q --upgrade openai # not to get the OpenAI proxies error: https://community.openai.com/t/error-with-openai-1-56-0-client-init-got-an-unexpected-keyword-argument-proxies/1040332/2

Audited 1 package in 50ms
Resolved 39 packages in 1.32s
Prepared 2 packages in 328ms
Installed 3 packages in 13ms
 + assemblyai==0.36.0
 + assemblyai-haystack==0.1.1
 + websockets==14.1


In [12]:
from getpass import getpass

ASSEMBLYAI_API_KEY = getpass("Enter AssemblyAI Api key: ")

### Running the `AssemblyAITranscriber` for a given audio file:

In [13]:
from assemblyai_haystack.transcriber import AssemblyAITranscriber

audio_file = "https://assembly.ai/story-of-santa-claus.mp3"

transcriber = AssemblyAITranscriber(api_key=ASSEMBLYAI_API_KEY)

result = transcriber.run(audio_file) 

In [14]:
print(result["transcription"][0].content)

The Story of Santa Claus There once lived a wealthy and rich man named Nicholas who lived in the North Pole. He was a kind, generous man. Nicholas enjoyed surprising people, and he often did things to make others happy. Nicholas had a special hobby. He would carve out pretty wooden toys in his free time. But no one ever saw these toys. It was Christmas time, and Nicholas wanted to do something special for the children around him. While sitting in his chair one morning, he suddenly had an idea. Nicholas was going to carve many beautiful toys for all the little boys and girls. Christmas presents. He would go climb up the chimney of all the houses and drop presents into the stockings that hung near the fireplace. Late that night, while everyone slept, Nicholas crept towards a nearby house and climbed to the top of the roof. He threw a toy into the chimney and quickly climbed back down. Nicholas did this to several houses, but after some time he became tired. He quickly went back to his ho

*“Task one, complete!”*

## Task 2: Summarizing Santa's Tale

Since the whole text is long to read. We'll have to code in a quick summary.


*Implementation:* Enable `summarization` feature of the `AssemblyAITranscriber` to make the story shorter.

**☝ TODO: Run the transcriber again and fill out the missing parts to enable summarization**

In [15]:
result = transcriber.run(audio_file, summarization=True)

# Printing Content of the Summarization
print(result["summarization"][0].content)

- The Story of Santa Claus. There once lived a wealthy and rich man named Nicholas who lived in the North Pole. He would climb up the chimney of all the houses and drop presents into the stockings that hung near the fireplace. The next morning, the children woke up and ran to the fireplace to find gifts.


## Task 3: Rewriting for Gen Z Kids

To reach a wider audience. We'll be using an LLM and a Haystack pipeline to transform the transcription into GenZ language.

### Setup

In [16]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI Api key: ")

' from getpass import getpass\nimport os\n\nos.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI Api key: ") '

### Pipeline Creation

In [18]:
from assemblyai_haystack.transcriber import AssemblyAITranscriber
from haystack import Pipeline
from haystack.utils import Secret
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator


template = """
Modify the tone of the transcribed story and transform it into Gen Z language.
Return only the modified story without a preface.

Transcript:
{{transcription}}
"""

prompt_builder = PromptBuilder(template=template)

transcriber = AssemblyAITranscriber(api_key=ASSEMBLYAI_API_KEY)

llm = OpenAIGenerator(model="gpt-4o-mini")

pipe = Pipeline()
pipe.add_component("transcriber", transcriber)
pipe.add_component("prompt_builder", prompt_builder)  
pipe.add_component("llm", llm) 


pipe.connect("transcriber.transcription", "prompt_builder.transcription")
pipe.connect("prompt_builder", "llm")

🚅 Components
  - transcriber: AssemblyAITranscriber
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - transcriber.transcription -> prompt_builder.transcription (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

*After the Pipeline was configured, Elf Patrick could run the whole pipeline and review the final output:*

☝Run the next cell.

In [ ]:
### Running the pipeline

In [19]:
audio_file = "https://assembly.ai/story-of-santa-claus.mp3"
res = pipe.run(
    {
        "transcriber": {
            "file_path": audio_file,
        }
    }
)

print(res["llm"]["replies"][0])

Yo, lemme tell you the tea about Santa Claus. So, there was this mega-rich dude named Nicholas, right? He was living it up in the North, totally vibin’. This guy had mad stacks, but instead of flexing on everyone, he decided to spread some holiday cheer instead. Total legend move. 

Instead of just chillin’ on his riches, he started this whole vibe of gifting people in his town. Like, imagine opening your door to find a surprise gift? Major wow. The man was just out there putting smiles on faces 'cause he knew what it was like to come from nothing.

As the years went by, people started calling him Santa, and he became the OG of gift-giving. Now, we all know him as this jolly guy in a red suit riding a sleigh led by reindeer—total squad goals. Every Christmas, he’d slide down chimneys dropping presents like it’s no big deal. 

So now, when holiday season rolls around, we’re all hyped about Santa, recognizing that it’s all about giving back and spreading the love. That’s the real vibe ch

## Conclusion

*Our fetched outputs are:*

1. *The Full Transcription - Transcribed using the [assemblyai-haystack integration](https://github.com/AssemblyAI/assemblyai-haystack) and the `AssemblyAITranscriber`*.
2. *The Summary - Also created with the `AssemblyAITranscriber` and `summarization` enabled.*
3. *The Gen Z Remix - Generated with a Haystack pipeline consisting of the transcriber, a prompt builder, and an LLM generator.*